In [ ]:
import json

# Load the original JSON data
with open('../Dataset/train.json', 'r') as json_file:
    data = json.load(json_file)

# Initialize lists for labels and paths
filtered_labels = []
filtered_paths = []

filtered_data = []
filtered_ids = []
video_paths = []

# Filter the data based on SENTENCE_DURATION between 2 and 3 seconds
for item in data:
    if 2.0 < item['SENTENCE_DURATION'] < 3.0:
        filtered_data.append(item)
        filtered_labels.append("<bos> " + item['SENTENCE_DESCRIPTION'] + " <eos>")  # Assuming 'SENTENCE_DESCRIPTION' contains labels
        filtered_paths.append(f"./3ktrain/features/{item['SENTENCE_NAME']}.npy")
        filtered_ids.append(item['SENTENCE_NAME'])
        video_paths.append(f"../../Dataset/train/videos/{item['SENTENCE_NAME']}.mp4")

# filtered_data = filtered_data[0:50]

print(len(filtered_data))

# Write the filtered labels and paths to separate JSON files
with open('3ktrain.json', 'w') as labels_file:
    json.dump(filtered_data, labels_file, indent=4)

In [ ]:
from mlflow import MLFlow

input_json_file = './3ktrain.json'
output_folder = './3ktrain/'
# processing_steps = ["crop", "reduce_noise", "add_noise", "rotate", "brightness", "contrast", "saturation"]

processor = MLFlow(input_json_file, output_folder)
vectors_df = processor.process_video_and_extract_features()

In [ ]:
import os
import json

videoId = []
videoSeq = []
videoSeq_raw = []
videoId_raw = []
features = []

In [ ]:
for i in range(0, len(filtered_data)):
    if os.path.exists(filtered_paths[i]):
        features.append(filtered_paths[i])
        videoId_raw.append(filtered_ids[i])
        videoSeq_raw.append(filtered_labels[i])

In [ ]:
import numpy as np

x_data = {}
for i in range(0, len(filtered_data)):
    f = np.load(filtered_paths[i])
    f = f.reshape(-1, int(np.prod(f.shape[1:])))
    if f.shape[1] == 487872:
        x_data[filtered_paths[i][19:-4]] = f
        videoId.append(videoId_raw[i])
        videoSeq.append(videoSeq_raw[i])

print(len(x_data))
print(len(videoId))
print(len(videoSeq))

In [ ]:
print(videoSeq[1], videoId[1])

In [ ]:
num_decoder_tokens = max(len(i) for i in filtered_labels)

In [ ]:
del filtered_labels, filtered_paths, filtered_data, filtered_ids, video_paths, videoSeq_raw, videoId_raw, features, f

In [ ]:

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=num_decoder_tokens)
tokenizer.fit_on_texts(videoSeq)
# word_index = tokenizer.word_index   
print ('Convert to index sequences.')
train_sequences = tokenizer.texts_to_sequences(videoSeq)
train_sequences = pad_sequences(train_sequences, padding='post',truncating='post')
train_sequences = np.array(train_sequences)
train_sequences = pad_sequences(train_sequences, padding='post',truncating='post')
print(train_sequences.shape)
# max_seq_length = train_sequences.shape[1]
filesize = len(train_sequences)

In [ ]:
print(filesize)

In [ ]:
from keras.utils import to_categorical
encoder_input_data = []
decoder_input_data = []
decoder_target_data = []
X_data = []
y_data = []
for idx in range(0, filesize):
    if len(x_data[videoId[idx]])==71:
        encoder_input_data.append(x_data[videoId[idx]])
        y = to_categorical(train_sequences[idx], num_decoder_tokens)
        decoder_input_data.append(y[:-1])
        decoder_target_data.append(y[1:])
encoder_input_data = np.array(encoder_input_data)
decoder_input_data = np.array(decoder_input_data)
decoder_target_data = np.array(decoder_target_data)

time_steps_decoder = decoder_input_data.shape[1]

print(encoder_input_data.shape)

In [ ]:
del x_data, train_sequences, filesize, videoSeq

In [ ]:
# from tensorflow.keras.models import Sequential

# from keras.layers import Input, LSTM, Dense

# simple_model = Sequential()

# simple_model.add(LSTM(32, input_shape=(60, 487872), return_sequences=True))

# simple_model.add(LSTM(32))

# simple_model.add(Dense(num_decoder_tokens, activation='softmax'))

# simple_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# from sklearn.model_selection import train_test_split


# encoder_input_data_train, encoder_input_data_test, decoder_input_data_train, decoder_input_data_test = train_test_split(encoder_input_data, decoder_target_data, test_size=0.2, random_state=42)


In [ ]:
# simple_model.fit(encoder_input_data_train, decoder_input_data_train, epochs=2, batch_size=1, validation_split=0.2)

In [ ]:
# scores = simple_model.evaluate(encoder_input_data_test, decoder_input_data_test, verbose=0)

# print("Test Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
from keras.layers import Input, LSTM, Dense, Permute, Reshape, Activation, Flatten

time_steps_encoder = 71
num_encoder_tokens = 487872
latent_dim = 8

encoder_inputs = Input(shape=(time_steps_encoder, num_encoder_tokens), name="encoder_inputs")
encoder = LSTM(latent_dim, return_state=True,return_sequences=True, name='endcoder_lstm')
_, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

In [ ]:
decoder_inputs = Input(shape=(time_steps_decoder, num_decoder_tokens), name= "decoder_inputs")
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, name='decoder_lstm')
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='relu', name='decoder_relu')
decoder_outputs = decoder_dense(decoder_outputs)

In [ ]:
from keras.models import Model

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary()

In [ ]:
from sklearn.model_selection import train_test_split

model.compile(metrics=['accuracy'], optimizer='adam', loss='categorical_crossentropy')

print(1)

encoder_input_data_train, encoder_input_data_test, decoder_input_data_train, decoder_input_data_test, decoder_target_data_train, decoder_target_data_test = train_test_split(encoder_input_data, decoder_input_data, decoder_target_data, test_size=0.2, random_state=42)

print(1)



In [ ]:
del encoder_input_data, decoder_input_data, decoder_target_data

In [ ]:
model.fit([encoder_input_data_train, decoder_input_data_train], decoder_target_data_train, batch_size=1, epochs=1)

In [ ]:
scores = model.evaluate([encoder_input_data_test, decoder_input_data_test], decoder_target_data_test, verbose=0)

print("Test Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
encoder_model = Model(encoder_inputs, encoder_states)
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)
encoder_model.summary()
decoder_model.summary()

In [ ]:
from keras.models import Model, load_model

inf_encoder_model = encoder_model
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
inf_decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)
inf_decoder_model.set_weights(decoder_model.get_weights())

In [ ]:
inf_encoder_model.save(os.path.join('./', 'inf_encoder_model.keras'))
inf_decoder_model.save(os.path.join('./', 'inf_decoder_model.keras'))

In [ ]:
# import functools, operator

# max_propablity = -1
# decode_seq = [] 

# def decode_sequence2bs(input_seq):
#     states_value = inf_encoder_model.predict(input_seq)
#     target_seq = np.zeros((1, 1, num_decoder_tokens))
#     target_seq[0, 0, tokenizer.word_index['bos']] = 1
#     beam_search(target_seq, states_value,[],[],0)
#     return decode_seq

# def beam_search(target_seq, states_value, prob,  path, lens):
#     global decode_seq
#     global max_propablity
#     node = 2
#     output_tokens, h, c = inf_decoder_model.predict(
#         [target_seq] + states_value)
#     output_tokens = output_tokens.reshape((num_decoder_tokens))
#     sampled_token_index = output_tokens.argsort()[-node:][::-1]
#     states_value = [h, c]
#     for i in range(node):
#         if sampled_token_index[i] == 0:
#             sampled_char = ''
#         else:
#             sampled_char = list(tokenizer.word_index.keys())[list(tokenizer.word_index.values()).index(sampled_token_index[i])]
#         MAX_LEN = 9
#         if(sampled_char != 'eos' and lens <= MAX_LEN):
#             p = output_tokens[sampled_token_index[i]]
#             if(sampled_char == ''):
#                 p = 1
#             prob_new = list(prob)
#             prob_new.append(p)
#             path_new = list(path)
#             path_new.append(sampled_char)
#             target_seq = np.zeros((1, 1, num_decoder_tokens))
#             target_seq[0, 0, sampled_token_index[i]] = 1.
#             beam_search(target_seq, states_value, prob_new, path_new, lens+1)
#         else:
#             p = output_tokens[sampled_token_index[i]]
#             prob_new = list(prob)
#             prob_new.append(p)
#             p = functools.reduce(operator.mul, prob_new, 1)
#             if(p > max_propablity):
#                 decode_seq = path
#                 max_propablity = p

# def decoded_sentence_tuning(decoded_sentence):
#     decode_str = []
#     filter_string = ['bos', 'eos']
#     unigram = {}
#     last_string = ""
#     for idx2, c in enumerate(decoded_sentence):
#         if c in unigram:
#             unigram[c] += 1
#         else:
#             unigram[c] = 1
#         if(last_string == c and idx2 > 0):
#             continue
#         if c in filter_string:
#             continue
#         if len(c) > 0:
#             decode_str.append(c)
#         if idx2 > 0:
#             last_string = c
#     return decode_str

# def test():
#     X_test = []
#     X_test_filename = []

#     for i in range(0, 1):
#         f = np.load(os.path.join(f"./train/features/{videoId[i]}.npy"))
#         X_test.append(f)
#         X_test_filename.append(videoId[i])
#     X_test = np.array(X_test)

#     with open(os.path.join('./', 'test_output.txt'), 'w') as file:
#         for idx, x in enumerate(X_test): 
#             file.write(X_test_filename[idx]+',')
#             decoded_sentence = decode_sequence2bs(x.reshape(-1, 48, 487872))
#             decode_str = decoded_sentence_tuning(decoded_sentence)
#             for d in decode_str:
#                 file.write(d + ' ')
#             file.write('\n')
#             max_propablity = -1

# test()

In [ ]:
# import numpy as np
# import os
# import functools
# import operator

# max_probability = -1
# decode_seq = []

# def decode_sequence_teacher_forcing(input_seq):
#     states_value = inf_encoder_model.predict(input_seq)
#     target_seq = np.zeros((1, 1, num_decoder_tokens))
#     target_seq[0, 0, tokenizer.word_index['bos']] = 1

#     # Initialize the output sequence
#     output_seq = np.zeros((1, 195, num_decoder_tokens))

#     for t in range(195):
#         output_tokens, h, c = inf_decoder_model.predict([target_seq] + states_value)
#         output_seq[0, t, :] = output_tokens[0, 0, :]

#         # Sample the next token from the output
#         sampled_token_index = np.argmax(output_tokens)
#         sampled_char = list(tokenizer.word_index.keys())[list(tokenizer.word_index.values()).index(sampled_token_index)]
        
#         if sampled_char == 'eos':
#             break

#         # Update the target sequence for the next time step
#         target_seq = np.zeros((1, 1, num_decoder_tokens))
#         target_seq[0, 0, sampled_token_index] = 1.

#         # Update states
#         states_value = [h, c]

#         print(f"Step {t + 1}: Predicted token: {sampled_char}")

#     return output_seq

# def test():
#     X_test = []
#     X_test_filename = []

#     for i in range(0, 10):
#         f = np.load(os.path.join(f"./train/features/{videoId[i]}.npy"))
#         X_test.append(f)
#         X_test_filename.append(videoId[i])
#     # X_test = np.array(X_test)

#     with open(os.path.join('./', 'test_output.txt'), 'w') as file:
#         for x in range(0, len(X_test)): 
#             file.write(X_test_filename[x]+',')
#             output_seq = decode_sequence_teacher_forcing(X_test[x].reshape(-1, 48, 487872))

#             # Process the output sequence
#             decoded_sentence = []
#             for t in range(195):
#                 sampled_token_index = np.argmax(output_seq[0, t, :])
#                 sampled_char = list(tokenizer.word_index.keys())[list(tokenizer.word_index.values()).index(sampled_token_index)]
#                 if sampled_char == 'eos':
#                     break
#                 if len(sampled_char) > 0:
#                     decoded_sentence.append(sampled_char)

#             decode_str = decoded_sentence_tuning(decoded_sentence)
#             for d in decode_str:
#                 file.write(d + ' ')
#             file.write('\n')

# test()
